In [1]:
import pickle

import pandas as pd
from sklearn.svm import SVC
from xgboost import XGBClassifier

from module.mymodule import best_parameters
from features import pipe_1, pipe_2, pipe_3, pipe_4, pipe_5, pipe_6, pipe_7,\
                     pipe_8, pipe_9, pipe_10

### オリジナルデータの呼び出し

In [2]:
df = pd.read_csv('./data/train.csv')

### 

### パイプライン候補からデータセットの作成

In [3]:
df = pd.read_csv('./data/train.csv')
to_pipe ={
            'df': df,
            'split_kwrg': {},
            'train_flg': True,
            'retrain': True,
            }

pipe_lines = [
            pipe_1,
            # pipe_2,
            # pipe_3,
            # pipe_4,
            # pipe_5,
            # pipe_6,
            # pipe_7,
            # pipe_8,
            pipe_9,
            pipe_10
            ]  # 特徴量候補を設定
data_set = {pipe.__name__: pipe(**to_pipe) for pipe in pipe_lines}

                                    Base                                   


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,56,1,155,342,1,150,1,3.0
1,55,0,130,394,0,150,0,0.0
2,47,1,110,0,1,120,1,0.0


                              AgeCAt Standard                              


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,3,1,155,342,1,150,1,3.0
1,3,0,130,394,0,150,0,0.0
2,2,1,110,0,1,120,1,0.0


                   StSlpoeCat CholestMean AgeCAt Standard                  


,Age,Sex,RestingBP,Cholesterol,FastingBS,MaxHR,ExerciseAngina,Oldpeak
0,3,1,155,342.000000,1,150,1,3.0
1,3,0,130,394.000000,0,150,0,0.0
2,2,1,110,243.414258,1,120,1,0.0


### グリットサーチで訓練したモデルの呼び出し

In [4]:
with open('./data/model.pkl', 'rb') as f:
    trained_models = pickle.load(f)

### 最適なパラメータで再訓練

In [5]:
best_param = best_parameters(trained_models, pipe_lines)
retrain = [
            XGBClassifier,
            #SVC
            ]

retrained = {}
for re_tr in retrain:
    retrained[re_tr.__name__] = {}
    for pipe in pipe_lines:
        x, y = data_set[pipe.__name__]
        param = best_param[re_tr.__name__][pipe.__name__]
        model = re_tr(**param)
        model.fit(x.values, y.values.reshape(-1))
        retrained[re_tr.__name__][pipe.__name__] = model

with open('./data/retrained_model.pkl', 'wb') as f:
    pickle.dump(retrained, f)